In [4]:
import pandas as pd
import numpy as np
#import nba_api as nba
#from nba_api.stats.library.parameters import LeagueID  # helps with api hits
#from nba_api.stats.endpoints import LeagueDashPlayerStats
#from nba_api.stats.endpoints import ScoreboardV2
from nba_api.stats.endpoints import PlayerDashboardByGeneralSplits
from sqlalchemy import create_engine

from datetime import datetime
import time, json, requests
import nbaPlayerStatModeling as nbaModel

In [5]:
nba = nbaModel.playerStatModel(
        day_offset = 0, 
        season = '2024-25', 
        perMode = 'PerGame'
)

In [6]:
nba.get_teams_playing()


3  games today...


In [7]:
nba.get_teams_pace(
    base_url = 'https://stats.nba.com/stats/leaguedashteamstats?Conference=&DateFrom=&DateTo=&Division=&GameScope=&GameSegment=&Height=&ISTRound=&LastNGames=0&LeagueID=00&Location=&MeasureType=Advanced&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerGame&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season={sid}&SeasonSegment=&SeasonType=Regular%20Season&ShotClockRange=&StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision='
)

In [9]:
nba.df_games_long

,GAME_ID,GAME_DATE_EST,GAME_STATUS_TEXT,TEAM_TYPE,TEAM_ID,oppTeamId,PACE
0,0022400752,2025-02-09,,Home,1610612765,1610612766,99.69
1,0022400752,2025-02-09,,Away,1610612766,1610612765,98.27
2,0022400753,2025-02-09,,Home,1610612745,1610612761,98.72
3,0022400753,2025-02-09,,Away,1610612761,1610612745,99.99
4,0022400754,2025-02-09,,Home,1610612749,1610612755,100.10
5,0022400754,2025-02-09,,Away,1610612755,1610612749,97.29


In [10]:
nba.get_players_playing(
    url_base_nba_player_stat = 'https://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode={per}&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season={sid}&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID={tid}&TwoWay=0&VsConference=&VsDivision=&Weight=',
    minute_cutoff = 15.0
)

99 players returned from 6 teams, out of a total 6


In [ ]:
nba.get_player_home_adv(
    use_default=False
)

In [19]:
df = nba.df_players
df = df[df['MIN'] >= 15.0]


(61, 47)

In [22]:
df.head()

,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,...,FTA_DIFF,OREB_DIFF,DREB_DIFF,REB_DIFF,AST_DIFF,TOV_DIFF,STL_DIFF,BLK_DIFF,BLKA_DIFF,PTS_DIFF
1,1641709,Ausar Thompson,1610612765,DET,20.6,3.5,6.8,0.515,0.2,1.1,...,0.283883,-0.150769,0.223558,0.118410,0.429980,0.033445,0.258974,0.418803,0.346154,0.096660
3,1630595,Cade Cunningham,1610612765,DET,35.7,9.6,21.1,0.453,2.3,6.4,...,0.207677,-0.916667,0.207134,0.104303,0.075590,-0.185036,-0.140873,0.433712,0.487403,0.010909
6,1630191,Isaiah Stewart,1610612765,DET,20.2,2.3,4.3,0.529,0.2,0.7,...,0.304348,0.273292,0.002389,0.105590,0.102123,-0.333333,0.165217,-0.419130,-0.739130,0.319471
7,1631093,Jaden Ivey,1610612765,DET,29.9,6.3,13.8,0.460,2.1,5.1,...,0.088435,0.000000,0.111111,0.076923,0.224242,-0.078431,-0.200000,-0.166667,0.090909,-0.042071
8,1631105,Jalen Duren,1610612765,DET,25.6,4.5,6.4,0.698,0.0,0.0,...,-0.274182,0.328932,0.104390,0.191011,0.163871,-0.045333,-0.234286,0.073103,0.253333,0.188211


In [31]:
conn_str = nbaModel.connect_to_database()

In [ ]:
# Define query with named placeholders
query = "SELECT * FROM my_table WHERE column_name = :value"

# Define parameters as a dictionary
params = {'value': 'some_value'}

# Execute query with parameters
df = pd.read_sql(query, engine, params=params)

'2025-02-09'

In [ ]:
query = """
    SELECT p.nbaId PLAYER_ID, p.player, o.prop, o.line, o.oOdds, o.uOdds  
    FROM odds o
    JOIN players p ON o.playerId = p.actnetId
    WHERE date = :game_date ;
""" #
props = pd.read_sql(
    sql = query, 
    con = conn_str,
    params = {'game_date':datetime.today().strftime('%Y-%m-%d')}
)


In [41]:
# Create SQLAlchemy engine
engine = create_engine(conn_str)

# Query with positional placeholder
query = """
    SELECT p.nbaId PLAYER_ID, p.player, o.prop, o.line, o.oOdds, o.uOdds  
    FROM odds o
    JOIN players p ON o.playerId = p.actnetId
    WHERE DATE(o.date) = %s;
"""

# Use context manager to handle connection lifecycle
with engine.connect() as connection:
    props = pd.read_sql(
        sql=query, 
        con=connection,  
        params=(datetime.today().strftime('%Y-%m-%d'),)  # Ensure tuple format
    )


NameError: name 'engine' is not defined